<a href="https://colab.research.google.com/github/DavidScience/AB_InBev_/blob/main/%5B4%5D_Desaf%C3%ADo_AB_InBev_DF_Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [0] Librerías necesarias

In [1]:
# Conectamos Google Drive con Google Colab
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Librerías necesarias para el desarrollo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import multiprocessing as mp
from joblib import Parallel, delayed
import pickle

## [1] Lectura de datos

In [3]:
# Lectura de datos preparados para el entrenamiento del algoritmo
datos = pd.read_csv('/gdrive/My Drive/AB_InBev/datos/data_test.csv')

In [4]:
# Inspección de las fechas más antiguas y recientes
print(datos["Date"].min())
print(datos["Date"].max())

2022-07-01
2022-07-30


In [5]:
# Convertimos el campo Date a fecha
datos['Date'] = pd.to_datetime(datos['Date'])

In [6]:
# Inspección de las fechas más antiguas y recientes
print(datos["Date"].min())
print(datos["Date"].max())

2022-07-01 00:00:00
2022-07-30 00:00:00


In [7]:
# Inspeccionamos los primeros registros
datos.head()

,Date,Account_id,Product_id,Category,Quantity,Year,Month
0,2022-07-01,33217788,8206,Gaseosas,240,2022,7
1,2022-07-01,33217788,8296,Gaseosas,80,2022,7
2,2022-07-01,33217788,8306,Gaseosas,80,2022,7
3,2022-07-01,33217788,8312,Gaseosas,80,2022,7
4,2022-07-01,33217788,8322,Gaseosas,320,2022,7


## [2] Preparación de los datos

In [8]:
# Creamos un indicador que concatena los indicadores de cliente y producto
datos["product"] = datos['Account_id'].astype("str") + '_' + datos['Product_id'].astype("str")

In [9]:
# Filtramos los datos a ser modelados
datos_mod = datos[["Date", "product", "Quantity"]]

In [10]:
# Agregamos los datos a nivel día
datos_groupby = datos_mod.groupby(["Date", "product"]) \
    .agg({"Quantity": "sum"}) \
    .rename(columns={"sum(link_cards)": "sum_link_cards"}).reset_index().sort_values(by = "Quantity", ascending = False)

In [11]:
# Filtramos los datos a ser modelados
datos_groupby_quan = datos_groupby[["Date", "product", "Quantity"]]

In [12]:
# Creamos una copia de los datos a ser modelados
features_names = ['Quantity']
datos_groupby_2 = datos_groupby_quan.copy()

In [13]:
# Creamos variables autoregresoras
for i in tqdm(range(1, 31)):
    expressions = {column: f"{column}_lag_{i}" for column in features_names}
    temporary_df = datos_groupby_2.assign(Date=lambda x: x["Date"] + pd.Timedelta(days=i)) \
                               .rename(columns=expressions)
    datos_groupby_quan = datos_groupby_quan.merge(temporary_df, on=["Date", "product"], how="left")

100%|██████████| 30/30 [00:02<00:00, 10.70it/s]


In [14]:
# Analizamos los datos iniciales
datos_groupby_quan.head()

,Date,product,Quantity,Quantity_lag_1,Quantity_lag_2,Quantity_lag_3,Quantity_lag_4,Quantity_lag_5,Quantity_lag_6,Quantity_lag_7,...,Quantity_lag_21,Quantity_lag_22,Quantity_lag_23,Quantity_lag_24,Quantity_lag_25,Quantity_lag_26,Quantity_lag_27,Quantity_lag_28,Quantity_lag_29,Quantity_lag_30
0,2022-07-21,34472346_14014,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-07-01,33229500_15354,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-21,33252756_8156,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-07-21,33250839_34372,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-07-21,33227463_34366,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Separamos los datos para el entrenamiento y validación de los algoritmos
test  = datos_groupby_quan.reset_index(drop=True).fillna(-1)

In [19]:
# Inspeccionamos los primeros registros de los datos de testeo
test.head(3)

,Date,product,Quantity,Quantity_lag_1,Quantity_lag_2,Quantity_lag_3,Quantity_lag_4,Quantity_lag_5,Quantity_lag_6,Quantity_lag_7,...,Quantity_lag_21,Quantity_lag_22,Quantity_lag_23,Quantity_lag_24,Quantity_lag_25,Quantity_lag_26,Quantity_lag_27,Quantity_lag_28,Quantity_lag_29,Quantity_lag_30
0,2022-07-21,34472346_14014,1200,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,2022-07-01,33229500_15354,1200,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2022-07-21,33252756_8156,1200,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [22]:
# Scoring del último día del mes de Julio
data_scoring = test[(test['Date']=='2022-07-30 00:00:00')]

In [23]:
# Validamos la volumetría de los datos
data_scoring.shape

(4878, 33)

In [24]:
# Inspeccionamos los primeros cinco registros
data_scoring.head(5)

,Date,product,Quantity,Quantity_lag_1,Quantity_lag_2,Quantity_lag_3,Quantity_lag_4,Quantity_lag_5,Quantity_lag_6,Quantity_lag_7,...,Quantity_lag_21,Quantity_lag_22,Quantity_lag_23,Quantity_lag_24,Quantity_lag_25,Quantity_lag_26,Quantity_lag_27,Quantity_lag_28,Quantity_lag_29,Quantity_lag_30
803,2022-07-30,40635321_17466,1173,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
805,2022-07-30,36756690_14016,1173,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1703,2022-07-30,33222003_15354,1122,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,790.0,-1.0,-1.0
1704,2022-07-30,36615003_15354,1122,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2002,2022-07-30,37607805_15354,1100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [30]:
# Almacenamos el datos de las variables
vars = data_scoring.columns.to_list()
vars = vars[2:-1]

In [31]:
# Filtramos las variables predictoras
X_test  = data_scoring[vars].values

In [34]:
# Lectura del modelo
filename = '/gdrive/My Drive/AB_InBev/models/finalized_model.sav'

# Cargando el modelo
loaded_model = pickle.load(open(filename, 'rb'))

## [4] Scoring del algoritmo

In [36]:
# Scoring de los datos de testeo
scoreo = loaded_model.predict(pd.DataFrame(X_test), ntree_limit=loaded_model.best_iteration)

[LightGBM] [Warning] Unknown parameter: ntree_limit


In [39]:
# Incluímos la variable predicha
data_scoring["y_pred"] = scoreo

<ipython-input-39-61eca8e3931a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_scoring["y_pred"] = scoreo


In [40]:
# Lectura de los primeros 5 registros
data_scoring.head(5)

,Date,product,Quantity,Quantity_lag_1,Quantity_lag_2,Quantity_lag_3,Quantity_lag_4,Quantity_lag_5,Quantity_lag_6,Quantity_lag_7,...,Quantity_lag_22,Quantity_lag_23,Quantity_lag_24,Quantity_lag_25,Quantity_lag_26,Quantity_lag_27,Quantity_lag_28,Quantity_lag_29,Quantity_lag_30,y_pred
803,2022-07-30,40635321_17466,1173,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,656.867577
805,2022-07-30,36756690_14016,1173,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,951.411464
1703,2022-07-30,33222003_15354,1122,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,790.0,-1.0,-1.0,772.467871
1704,2022-07-30,36615003_15354,1122,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,656.867577
2002,2022-07-30,37607805_15354,1100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,656.867577


In [41]:
# Exportamos los datos
data_scoring.to_csv('/gdrive/My Drive/AB_InBev/datos/data_scoring_df.csv',index=False)